In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as k
import data
import os
import csv
import re
from string import digits

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.corpus import wordnet

In [4]:
# Define the file we want to read
MSFT_Sentiment_FILE = 'clean_msft_tweets_4m_v'
data_path = os.path.join(os.getcwd(), MSFT_Sentiment_FILE)

data_in = pd.read_csv(data_path, delimiter = ',', index_col=0)
print(data_in.info())

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (5,6,12,19,20,26,33,34,40,47,54,55,68,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13557 entries, 0 to 13556
Columns: 113 entries, 17/03/2016 to 17/11/2015
dtypes: object(113)
memory usage: 11.8+ MB
None


In [5]:
data_in.sample(n=5)

,17/03/2016,16/03/2016,15/03/2016,14/03/2016,13/03/2016,12/03/2016,11/03/2016,10/03/2016,09/03/2016,08/03/2016,...,28/11/2015,25/11/2015,24/11/2015,23/11/2015,22/11/2015,21/11/2015,20/11/2015,19/11/2015,18/11/2015,17/11/2015
9972,court,Tz,thoughts,Microsoft,NaN,NaN,W,Azure,Defender,pushed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dicht,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10249,NaN,HQ,Microsoft,less,NaN,NaN,Ballmer,Android,Helped,Microsoft,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12416,NaN,NaN,NaN,women,NaN,NaN,Design,NaN,NaN,W,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9145,Amazing,Micr,GB,need,NaN,NaN,numbers,microsoft,OfficeProfessionalPlus,Alzacz,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#TO get Bags of words
from collections import Counter
date_BOW = {}
for i in data_in.columns:
    cnt = Counter()
    for j in data_in[i]:
        cnt[j] += 1
    date_BOW[i] = cnt

In [7]:
print(data_in.columns)

Index(['17/03/2016', '16/03/2016', '15/03/2016', '14/03/2016', '13/03/2016',
       '12/03/2016', '11/03/2016', '10/03/2016', '09/03/2016', '08/03/2016',
       ...
       '28/11/2015', '25/11/2015', '24/11/2015', '23/11/2015', '22/11/2015',
       '21/11/2015', '20/11/2015', '19/11/2015', '18/11/2015', '17/11/2015'],
      dtype='object', length=113)


In [8]:
print(type(date_BOW['17/03/2016']))
print(len(date_BOW['16/03/2016']))
print(len(date_BOW['15/03/2016']))

<class 'collections.Counter'>
4608
4619


In [9]:
date_BOW['06/12/2015'].keys() #The NaN should be removed!

dict_keys(['A', 'Reddit', 'user', 'creates', 'playable', 'XCOM', 'game', 'Microsoft', 'Excel', 'via', 'yashparikh', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,

In [10]:
for date in date_BOW.keys():
    j = 0
    del_list = []
    for i in date_BOW[date].keys():
        if isinstance(i, float):
            j+=1
            del_list.append(i)
    for k in range(len(del_list)):        
        del date_BOW[date][del_list[k]]
    print(date,'done! cleared: ', j)

17/03/2016 done! cleared:  1
16/03/2016 done! cleared:  1
15/03/2016 done! cleared:  1
14/03/2016 done! cleared:  1
13/03/2016 done! cleared:  5366
12/03/2016 done! cleared:  5366
11/03/2016 done! cleared:  1
10/03/2016 done! cleared:  1
09/03/2016 done! cleared:  1
08/03/2016 done! cleared:  0
07/03/2016 done! cleared:  1
06/03/2016 done! cleared:  5366
05/03/2016 done! cleared:  1
04/03/2016 done! cleared:  1
03/03/2016 done! cleared:  1
02/03/2016 done! cleared:  1
01/03/2016 done! cleared:  1
29/02/2016 done! cleared:  1
28/02/2016 done! cleared:  5366
27/02/2016 done! cleared:  5366
26/02/2016 done! cleared:  1
25/02/2016 done! cleared:  1
24/02/2016 done! cleared:  1
23/02/2016 done! cleared:  1
22/02/2016 done! cleared:  1
21/02/2016 done! cleared:  5366
20/02/2016 done! cleared:  1
19/02/2016 done! cleared:  1
18/02/2016 done! cleared:  1
17/02/2016 done! cleared:  1
16/02/2016 done! cleared:  1
15/02/2016 done! cleared:  1
14/02/2016 done! cleared:  5366
13/02/2016 done! clear

In [11]:
#Set all nan -> '0'
def set_nan_0(tweet_df_col):
    for i in range(tweet_df_col.shape[0]):
        if isinstance(tweet_df_col[i], float):
            tweet_df_col[i] = ''
    return tweet_df_col

In [12]:
# Remove NaN
del_list = []
for date in date_BOW.keys():
    for key in date_BOW[date].keys():
        if isinstance(key, float):
            del_list.append(key)
    for i in range(len(del_list)):
        del date_BOW[date][i]
    print(date, 'Done!')


17/03/2016 Done!
16/03/2016 Done!
15/03/2016 Done!
14/03/2016 Done!
13/03/2016 Done!
12/03/2016 Done!
11/03/2016 Done!
10/03/2016 Done!
09/03/2016 Done!
08/03/2016 Done!
07/03/2016 Done!
06/03/2016 Done!
05/03/2016 Done!
04/03/2016 Done!
03/03/2016 Done!
02/03/2016 Done!
01/03/2016 Done!
29/02/2016 Done!
28/02/2016 Done!
27/02/2016 Done!
26/02/2016 Done!
25/02/2016 Done!
24/02/2016 Done!
23/02/2016 Done!
22/02/2016 Done!
21/02/2016 Done!
20/02/2016 Done!
19/02/2016 Done!
18/02/2016 Done!
17/02/2016 Done!
16/02/2016 Done!
15/02/2016 Done!
14/02/2016 Done!
13/02/2016 Done!
12/02/2016 Done!
11/02/2016 Done!
10/02/2016 Done!
09/02/2016 Done!
08/02/2016 Done!
07/02/2016 Done!
06/02/2016 Done!
05/02/2016 Done!
04/02/2016 Done!
03/02/2016 Done!
02/02/2016 Done!
01/02/2016 Done!
31/01/2016 Done!
30/01/2016 Done!
29/01/2016 Done!
28/01/2016 Done!
27/01/2016 Done!
26/01/2016 Done!
25/01/2016 Done!
24/01/2016 Done!
23/01/2016 Done!
22/01/2016 Done!
21/01/2016 Done!
20/01/2016 Done!
19/01/2016 Don

In [13]:
print(date_BOW.keys()); print()
# print(date_BOW['16/03/2016'].keys())
#If seeing nan, use the box above

dict_keys(['17/03/2016', '16/03/2016', '15/03/2016', '14/03/2016', '13/03/2016', '12/03/2016', '11/03/2016', '10/03/2016', '09/03/2016', '08/03/2016', '07/03/2016', '06/03/2016', '05/03/2016', '04/03/2016', '03/03/2016', '02/03/2016', '01/03/2016', '29/02/2016', '28/02/2016', '27/02/2016', '26/02/2016', '25/02/2016', '24/02/2016', '23/02/2016', '22/02/2016', '21/02/2016', '20/02/2016', '19/02/2016', '18/02/2016', '17/02/2016', '16/02/2016', '15/02/2016', '14/02/2016', '13/02/2016', '12/02/2016', '11/02/2016', '10/02/2016', '09/02/2016', '08/02/2016', '07/02/2016', '06/02/2016', '05/02/2016', '04/02/2016', '03/02/2016', '02/02/2016', '01/02/2016', '31/01/2016', '30/01/2016', '29/01/2016', '28/01/2016', '27/01/2016', '26/01/2016', '25/01/2016', '24/01/2016', '23/01/2016', '22/01/2016', '21/01/2016', '20/01/2016', '19/01/2016', '18/01/2016', '17/01/2016', '16/01/2016', '15/01/2016', '14/01/2016', '13/01/2016', '12/01/2016', '11/01/2016', '10/01/2016', '09/01/2016', '08/01/2016', '07/01/20

In [14]:
# TextBlob sentiment analysis

In [1]:
from textblob import TextBlob
#https://textblob.readthedocs.io/en/dev/quickstart.html 
# testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
testimonial = TextBlob("great")
testimonial.sentiment

Sentiment(polarity=0.8, subjectivity=0.75)

In [16]:
Score_daily = {}
for date in date_BOW.keys():
    polarity_score_pos = 0; polarity_score_neg = 0; polarity_score_neu = 0; subjective_score = 0
    pos_word_num = 0; neg_word_num = 0; neu_word_num = 0;
    for word, count in date_BOW[date].items():
        testimonial = TextBlob(word)
        raw_sent_score, raw_subject_score = testimonial.sentiment
#         raw_subject_score = testimonial.sentiment[1]
        if raw_sent_score > 0:
            pos_word_num += count
            polarity_score_pos += raw_sent_score * count
        if raw_sent_score < 0:
            neg_word_num += count
            polarity_score_neg += raw_sent_score * count
        if raw_sent_score == 0:
            neu_word_num += count
            polarity_score_neu += raw_sent_score * count
        
        subjective_score += raw_subject_score * count
        Score_daily[date] = (polarity_score_pos, polarity_score_neg, polarity_score_neu,subjective_score, pos_word_num, neg_word_num, neu_word_num)
    print(date, 'Done!')

17/03/2016 Done!
16/03/2016 Done!
15/03/2016 Done!
14/03/2016 Done!
13/03/2016 Done!
12/03/2016 Done!
11/03/2016 Done!
10/03/2016 Done!
09/03/2016 Done!
08/03/2016 Done!
07/03/2016 Done!
06/03/2016 Done!
05/03/2016 Done!
04/03/2016 Done!
03/03/2016 Done!
02/03/2016 Done!
01/03/2016 Done!
29/02/2016 Done!
28/02/2016 Done!
27/02/2016 Done!
26/02/2016 Done!
25/02/2016 Done!
24/02/2016 Done!
23/02/2016 Done!
22/02/2016 Done!
21/02/2016 Done!
20/02/2016 Done!
19/02/2016 Done!
18/02/2016 Done!
17/02/2016 Done!
16/02/2016 Done!
15/02/2016 Done!
14/02/2016 Done!
13/02/2016 Done!
12/02/2016 Done!
11/02/2016 Done!
10/02/2016 Done!
09/02/2016 Done!
08/02/2016 Done!
07/02/2016 Done!
06/02/2016 Done!
05/02/2016 Done!
04/02/2016 Done!
03/02/2016 Done!
02/02/2016 Done!
01/02/2016 Done!
31/01/2016 Done!
30/01/2016 Done!
29/01/2016 Done!
28/01/2016 Done!
27/01/2016 Done!
26/01/2016 Done!
25/01/2016 Done!
24/01/2016 Done!
23/01/2016 Done!
22/01/2016 Done!
21/01/2016 Done!
20/01/2016 Done!
19/01/2016 Don

In [17]:
len(Score_daily)

113

In [18]:
# Sentiment_dict = {}
# for date in date_BOW.keys():
#     testimonial = TextBlob[date_BOWp[date]]

In [19]:
Score_daily #Date-Polarity-Subjectivity

{'17/03/2016': (123.06317099567086,
  -32.115476190476194,
  0.0,
  253.6716955266956,
  317,
  101,
  9692),
 '16/03/2016': (148.81584415584413,
  -32.87400793650794,
  0.0,
  296.8566305916304,
  378,
  103,
  11191),
 '15/03/2016': (145.52472943722947,
  -42.53095238095238,
  0.0,
  291.667803030303,
  377,
  103,
  11410),
 '14/03/2016': (151.11238095238085,
  -43.30119047619049,
  0.0,
  313.1771212121213,
  392,
  102,
  12130),
 '13/03/2016': (100.79397186147185,
  -25.982142857142865,
  0.0,
  200.2286291486291,
  258,
  91,
  7384),
 '12/03/2016': (119.93214285714282,
  -30.362316849816857,
  0.0,
  221.727140914641,
  283,
  88,
  7746),
 '11/03/2016': (155.17754329004322,
  -28.463888888888892,
  0.0,
  292.5105699855698,
  401,
  85,
  12548),
 '10/03/2016': (152.8780735930736,
  -27.904761904761905,
  0.0,
  314.10778499278496,
  414,
  84,
  10861),
 '09/03/2016': (163.0569805194805,
  -32.29537545787547,
  0.0,
  365.057831057831,
  456,
  98,
  10745),
 '08/03/2016': (1

In [20]:
pd_Score_daily = pd.DataFrame.from_dict(Score_daily) #Date-Polarity-Subjectivity->DataFrame

In [21]:
(pd_Score_daily.head())


,17/03/2016,16/03/2016,15/03/2016,14/03/2016,13/03/2016,12/03/2016,11/03/2016,10/03/2016,09/03/2016,08/03/2016,...,28/11/2015,25/11/2015,24/11/2015,23/11/2015,22/11/2015,21/11/2015,20/11/2015,19/11/2015,18/11/2015,17/11/2015
0,123.063171,148.815844,145.524729,151.112381,100.793972,119.932143,155.177543,152.878074,163.056981,176.423009,...,0.0,0.0,0.4,1.233333,0.0,0.0,0.0,0.136364,0.0,0.1
1,-32.115476,-32.874008,-42.530952,-43.301190,-25.982143,-30.362317,-28.463889,-27.904762,-32.295375,-42.427778,...,0.0,0.0,-0.1,-0.400000,0.0,0.0,0.0,0.000000,0.0,-0.4
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,253.671696,296.856631,291.667803,313.177121,200.228629,221.727141,292.510570,314.107785,365.057831,393.874856,...,0.0,0.0,1.0,2.800000,0.0,0.0,0.0,0.500000,0.0,0.7
4,317.000000,378.000000,377.000000,392.000000,258.000000,283.000000,401.000000,414.000000,456.000000,476.000000,...,0.0,0.0,1.0,2.000000,0.0,0.0,0.0,1.000000,0.0,1.0


In [23]:
#By TextBlob sentiment
pd_Score_daily.to_csv('msft_11_15_to_03_16_news.csv')

In [24]:
# xs = range(0,len(date_BOW['17/03/2016']))
# figure(figsize=(16, 7), dpi=80, facecolor='w', edgecolor='k')
# barplt_col_sum = plt.bar(xs, date_BOW['17/03/2016'].values(), color='g', width = 0.8, align = 'center')
# plt.xticks(rotation=-90)
# # plt.yticks(np.arange(0 , 15000, step = 1000))
# plt.title('Unique word count on the 17/03/2016')
# plt.xlabel('Date')
# plt.ylabel('Word count')
# plt.savefig('date_volumne4_2.pdf')

In [25]:
syns = wordnet.synsets('program')
#Synset
print(syns[0].name())
#Definition
print(syns[0].definition())
#Example
print(syns[0].examples())


plan.n.01
a series of steps to be carried out or goals to be accomplished
['they drew up a six-step plan', 'they discussed plans for a new bond issue']


In [26]:
#Synonymns
word = 'good'

def Synonymns_gen(word):
    Synonymns=[]
    Antonyms=[]
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
#             print('l',l) #Show all lemmas, try 'good'
            Synonymns.append(l.name())
            if l.antonyms():
                Antonyms.append(l.antonyms()[0].name())
    return (Synonymns, Antonyms)
                
(Synonymns, Antonyms) = Synonymns_gen(word)           
print(set(Synonymns))
print(set(Antonyms))

{'adept', 'full', 'safe', 'in_effect', 'respectable', 'just', 'soundly', 'ripe', 'proficient', 'near', 'secure', 'undecomposed', 'upright', 'good', 'well', 'in_force', 'serious', 'dear', 'practiced', 'goodness', 'effective', 'trade_good', 'beneficial', 'sound', 'expert', 'estimable', 'thoroughly', 'honest', 'skillful', 'salutary', 'skilful', 'commodity', 'unspoiled', 'honorable', 'dependable', 'right', 'unspoilt'}
{'evilness', 'bad', 'ill', 'evil', 'badness'}


In [27]:
#Word similarity

def compute_word_sim(W1, W2):
    sim = (W1.wup_similarity(W2))
    return sim

W1 = wordnet.synsets('ship')[0]
W2 = wordnet.synsets('boat')[0]
sim1 = compute_word_sim(W1, W2)
print(sim1)
W3 = wordnet.synsets('cat')[0]
W4 = wordnet.synsets('dog')[0]
sim2 = compute_word_sim(W3, W4)
print(sim2)
W5 = wordnet.synsets('Black')[0]
W6 = wordnet.synsets('back')[0]
sim3 = compute_word_sim(W5, W6)
print(sim3)
W7 = wordnet.synsets('radiation')[0]
W8 = wordnet.synsets('light')[0]
sim4 = compute_word_sim(W7, W8)
print(sim4)
W9 = wordnet.synsets('lesbian')[0]
W10 = wordnet.synsets('gay')[0]
sim3 = compute_word_sim(W9, W10)
print(sim3)
W11 = wordnet.synsets('finger')[0]
W12 = wordnet.synsets('hand')[0]
sim4 = compute_word_sim(W11, W12)
print(sim4)

0.9090909090909091
0.8571428571428571
0.14285714285714285
0.8421052631578947
0.7058823529411765
0.875


In [28]:
print(syns[0].lemmas())
#Just name
print(syns[0].lemmas()[0].name())

[Lemma('plan.n.01.plan'), Lemma('plan.n.01.program'), Lemma('plan.n.01.programme')]
plan


In [29]:
from nltk.stem import *
from nltk.corpus import sentiwordnet as swn
# SentiNet 
#https://stackoverflow.com/questions/33959459/usage-of-nltk-sentiwordnet-with-python
# Sentiment Symposium Tutorial: Lexicons (Useful guide)
# http://sentiment.christopherpotts.net/lexicons.html#resources
# http://ontotext.fbk.eu/sentiwn.html

In [77]:
W2 = list(swn.senti_synsets('sad'))
print(W2)
w2_0 = W2[0]
w2_1 = W2[1]
w2_2 = W2[2]

[SentiSynset('sad.a.01'), SentiSynset('sad.s.02'), SentiSynset('deplorable.s.01')]


In [78]:
W1 = list(swn.senti_synsets('good'))
print(W1[:5])
w1_0 = W1[0]
w1_1 = W1[1]
w1_2 = W1[2]

[SentiSynset('good.n.01'), SentiSynset('good.n.02'), SentiSynset('good.n.03'), SentiSynset('commodity.n.01'), SentiSynset('good.a.01')]


In [81]:
print(); print(w1_0); print(w1_0)
print('pos: ', w1_0.pos_score())
print('neg: ', w1_0.neg_score())

print(); print(w1_1); print(W1_1)
print('pos: ', w1_1.pos_score())
print('neg: ', w1_1.neg_score())

print(); print(w1_2); print(w1_2)
print('pos: ', w1_2.pos_score())
print('neg: ', w1_2.neg_score())


<good.n.01: PosScore=0.5 NegScore=0.0>
<good.n.01: PosScore=0.5 NegScore=0.0>
pos:  0.5
neg:  0.0

<good.n.02: PosScore=0.875 NegScore=0.0>
<good.n.02: PosScore=0.875 NegScore=0.0>
pos:  0.875
neg:  0.0

<good.n.03: PosScore=0.625 NegScore=0.0>
<good.n.03: PosScore=0.625 NegScore=0.0>
pos:  0.625
neg:  0.0


In [82]:
print(); print(w2_0); print(w2_0)
print('pos: ', w2_0.pos_score())
print('neg: ', w2_0.neg_score())

print(); print(w2_1); print(w2_1)
print('pos: ', w2_1.pos_score())
print('neg: ', w2_1.neg_score())

print(); print(w2_2); print(w2_2)
print('pos: ', w2_2.pos_score())
print('neg: ', w2_2.neg_score())


<sad.a.01: PosScore=0.125 NegScore=0.75>
<sad.a.01: PosScore=0.125 NegScore=0.75>
pos:  0.125
neg:  0.75

<sad.s.02: PosScore=0.0 NegScore=0.25>
<sad.s.02: PosScore=0.0 NegScore=0.25>
pos:  0.0
neg:  0.25

<deplorable.s.01: PosScore=0.0 NegScore=1.0>
<deplorable.s.01: PosScore=0.0 NegScore=1.0>
pos:  0.0
neg:  1.0


In [ ]:
#OpinionFinder-

